<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/LangChain_Complete_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install transformers
!pip install openai
!pip install datasets
!pip install langchain
!pip install huggingface

In [36]:
from google.colab import userdata
hf_key=userdata.get('huggingface')
OPENAI_API_KEY=userdata.get('openai')

In [37]:
import os
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN']=hf_key

In [38]:
#openai
#example1

from langchain.llms import OpenAI
llm=OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.9)
print(llm)

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.9, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256}


In [39]:
text="Suggest me 5 names for italian resturants"
print(llm.invoke(text))



1. Trattoria Bellissimo
2. Cucina Rustica
3. La Piazza Bella
4. Ristorante Amore
5. L'Angolo di Roma


In [40]:
from langchain import HuggingFaceHub

In [41]:
llm=HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'temperature':0.5})

In [42]:
llm('Translate English to German: I Come from india')

'Translate English to German: I Come from india\n\nIch komme aus Indien.\n\nHow do you say "I come from India" in German?\n\nTo say "I come from India" in German, you can say "Ich komme aus Indien." This sentence follows the standard structure for expressing where someone comes from in German, using the first-person singular pronoun "Ich" (I), the verb "kommen" (to come), and the prepositional phrase "aus Ind'

In [43]:
#prompt template
from langchain.prompts import PromptTemplate
llm=OpenAI(temperature=0.8)

prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template='I want to open a resturant for {cuisine} food. Suggest a fancy name for this'
)
p=prompt_template_name.format(cuisine='German')
print(p)

I want to open a resturant for German food. Suggest a fancy name for this


In [44]:
llm.invoke(p)

'\n\n"Königliche Küche" (Royal Cuisine)'

In [45]:
#chains
prompt=PromptTemplate(
    input_variables=['products'],
    template='what is a good name for a company that makes {products}')
p=prompt.format(products='socks')
print(p)


what is a good name for a company that makes socks


In [46]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm, prompt=prompt)
response=chain.run('socks')
print(response)



1. "Happy Feet Co."
2. "Soleful Socks"
3. "Socktopia"
4. "Toasty Toes"
5. "Sock Savvy"
6. "Footprints Co."
7. "Socktastic"
8. "Step by Step Socks"
9. "Cozy Feet Co."
10. "Socktopus"


In [47]:
llm=HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1')
prompt=PromptTemplate(
    input_variables=['Game'],
    template='Do you know about the {game}? write 1 line about it'
)

prompt1=PromptTemplate(
    input_variables=['Game'],
    template='Do you know about the {game}? write 1 line about it'
)


chain1=LLMChain(llm=llm, prompt=prompt)
chain2=LLMChain(llm=llm, prompt=prompt1)
response=chain.run('Overwatch')
print(response)



"Heroic Gaming Inc."


In [48]:
#Simple Sequential Chain
#will only show output of last chain
llm=HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1',
                   model_kwargs={'temperature':0.1,'max_seq_length':100})
prompt=PromptTemplate(
    input_variables=['city'],
    template='What is a good Resturant in {city}?'
)

prompt1=PromptTemplate(
    input_variables=['resturant'],
    template='Do you know about the {resturant}?'
)

chain1=LLMChain(llm=llm, prompt=prompt)
chain2=LLMChain(llm=llm, prompt=prompt1)

from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[chain1, chain2])
response=chain.run('Toronto')
print(response)

Do you know about the What is a good Resturant in Toronto?

Toronto is a city that is known for its diverse food scene. There are many great restaurants in Toronto, but here are a few that are particularly noteworthy:

1. Alo: This upscale French restaurant is located in the heart of downtown Toronto and is known for its innovative and delicious dishes.
2. Pai Northern Thai Kitchen: This casual restaurant serves up delicious and authentic Thai food, including a wide variety of curries, noodle?dishes, and stir-fries.
3. Terroni: This popular Italian restaurant has several locations throughout the city and is known for its delicious pizzas, pastas, and other Italian dishes.
4. The Carbon Bar: This upscale barbecue restaurant is located in the trendy Distillery District and is known for its delicious smoked meats and seafood.
5. Byblos: This Eastern Mediterranean restaurant is located in the Entertainment District and is


In [49]:
llm=OpenAI(temperature=0.1)
prompt_template1=PromptTemplate(
input_variables=['cuisine'],
template="I want to open a resturant for {cuisine} food. Suggest me 2 fancy names"
)
name_chain=LLMChain(llm=llm, prompt=prompt_template1, output_key='cuisine_suggested')

prompt_template2=PromptTemplate(
input_variables=['menu'],
template="Suggest some menu items also"
)
menu_chain=LLMChain(llm=llm, prompt=prompt_template2,output_key='menu_items')

from langchain.chains import SequentialChain
chain=SequentialChain(
    chains=[name_chain, menu_chain],
    input_variables=['cuisine'],
    output_variables=['cuisine_suggested','menu_items']
)

response=chain({'cuisine':'indian'})
print(response['cuisine'])
print(response['cuisine_suggested'])

indian


1. "Saffron Spice Bistro"
2. "Taj Mahal Tastes"


In [50]:
#Agents
!pip install google-search-results

In [51]:
import os
os.environ['SERPAPI_API_KEY']='9254413cc07017f0bc95f65e66ec43584a05ad37908171b283fbca77ead4383f'

In [52]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [53]:
!pip install wikipedia

In [54]:
tools=load_tools(['serpapi','llm-math'], llm=llm)
agent=initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run('GDP of US in 2010?')



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "GDP of US in 2010"
Observation: ["The U.S. is a country of 50 states covering a vast swath of North America, with Alaska in the northwest and Hawaii extending the nation’s presence into the Pacific Ocean. Major Atlantic Coast cities are New York, a global finance and culture center, and capital Washington, DC. Midwestern metropolis Chicago is known for influential architecture and on the west coast, Los Angeles' Hollywood is famed for filmmaking. ― Google", 'United States type: Country in North America.', 'United States entity_type: countries, locations, travel, competitions, military, travel.', 'United States kgmid: /m/09c7w0.', 'Gross domestic product of the United States from 1990 to 2023 (in billion current U.S. dollars ).', 'Manufacturing value addeda measure of an industrys contribution to GDP rose 11.2 percent, a sharp return to growth after two consecutive yea

'The GDP of the US in 2010 was $15,049,000 million.'

In [55]:
tools=load_tools(['wikipedia','llm-math'], llm=llm)
agent=initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run('GDP of US in 2010?')



> Entering new AgentExecutor chain...
 I should use the calculator to calculate the GDP
Action: Calculator
Action Input: 14.6 trillion * 2010
Observation: Answer: 2.9346e+16
Thought: I should convert the answer to a more readable format
Action: Calculator
Action Input: 2.9346e+16
Observation: Answer: 2.9346e+16
Thought: I should use Wikipedia to find the GDP of US in 2010
Action: Wikipedia
Action Input: GDP of US in 2010
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use the calculator to calculate the GDP
Action: Calculator
Action Input: 14.6 trillion * 2010
Observation: Answer: 2.9346e+16
Thought: I should convert the answer to a more readable format
Action: Calculator
Action Input: 2.9346e+16
Observation: Answer: 2.9346e+16
Thought: I now know the final answer
Final Answer: The GDP of US in 2010 was $14.6 trillion.

> Finished chain.


'The GDP of US in 2010 was $14.6 trillion.'

In [56]:
prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template='I want to open a resturant for {cuisine}. Suggest fancy name for this'
)
chain=LLMChain(llm=llm, prompt=prompt_template_name)
name=chain.run('mexican')
print(name)



"El Sabor de México" (The Flavor of Mexico)


In [57]:
#conversation buffer memory
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()
chain=LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name=chain.run('Mexican')
print(name)



"El Sabor de México" (The Flavor of Mexico)


In [58]:
name=chain.run('Indian')
print(name)



1. "Saffron Spice"
2. "Taj Mahal Bistro"
3. "Curry Kingdom"
4. "Naan & Nirvana"
5. "Masala Mansion"
6. "Chai Chalet"
7. "Bollywood Bites"
8. "Flavors of India"
9. "Mughal Palace"
10. "Samosa Symphony"


In [59]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor de México" (The Flavor of Mexico)
Human: Indian
AI: 

1. "Saffron Spice"
2. "Taj Mahal Bistro"
3. "Curry Kingdom"
4. "Naan & Nirvana"
5. "Masala Mansion"
6. "Chai Chalet"
7. "Bollywood Bites"
8. "Flavors of India"
9. "Mughal Palace"
10. "Samosa Symphony"


In [60]:
#conversationchain
from langchain.chains import ConversationChain
convo=ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [61]:
convo.run(' who won the first cricket worldcup?')

' The first Cricket World Cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final match by 17 runs. The tournament was hosted by England and was played in a round-robin format with eight teams participating. The West Indies team was led by Clive Lloyd and had players like Viv Richards, Gordon Greenidge, and Michael Holding. It was a historic moment for the West Indies as it was their first World Cup win.'

In [62]:
convo.run('How much is 5+5')

'  5+5 is equal to 10.'

In [63]:
convo.run(' who was the captain of the winning team?')

' The captain of the winning team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He led the West Indies team to two consecutive World Cup wins in 1975 and 1979. He was known for his aggressive and fearless leadership style and was considered one of the greatest captains in cricket history.'

**Conversation buffer window memory  
for allowing remembering specific contexts**

In [64]:

from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=2)
convo=ConversationChain(
    llm=llm,
    memory=memory
)
convo.run('who won the first world cup ? , give short response in 2 lines')

' The first World Cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match.'

In [65]:
convo.run('what is 5+5?')

' 5+5 is equal to 10.'

In [66]:
convo.run('what was the captain of the winning team?')

' The captain of the winning team, Uruguay, was Jose Nasazzi. He was a defender and played for the club Nacional.'

Document Loaders

In [67]:
!pip install pypdf

In [68]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader('/content/Resume_CV_up.pdf')
pages=loader.load()

In [ ]:
pages